In [1]:
# agents/ai_explanatory_agent.py
from huggingface_hub import InferenceClient
from utils.config import HUGGINGFACE_API_KEY 
import time

class AIExplanatoryAgent:
    def __init__(self):
        # Initialize with free Hugging Face model (using random available model)
        self.client = InferenceClient()
        self.template = """Turn these data points into insurance insights:
        
        Disaster Event: {disaster_event}
        Claim Surge: {claim_surge}
        Economic Factors: {economic_factors}
        
        Insight:"""
    
    def generate_insight(self, disaster_event, claim_surge, economic_factors):
        """
        Generates natural language explanations from structured data
        """
        prompt = self.template.format(
            disaster_event=disaster_event,
            claim_surge=claim_surge,
            economic_factors=economic_factors
        )
        
        try:
            # Get response from Hugging Face's free inference API
            response = self.client.text_generation(
                prompt,
                max_new_tokens=100,
                model="mistralai/Mistral-7B-Instruct-v0.2"  # Free model good for text generation
            )
            return response.strip()
        except Exception as e:
            print(f"Error generating insight: {e}")
            return "Unable to generate insight at this time"

# Example usage (you can test this)
if __name__ == "__main__":
    agent = AIExplanatoryAgent()
    
    sample_data = {
        "disaster_event": "Category 4 Hurricane in Florida",
        "claim_surge": "35% increase in auto claims",
        "economic_factors": "Unemployment rate up 2%"
    }
    
    insight = agent.generate_insight(**sample_data)
    print("Generated Insight:")
    print(insight)

Error generating insight: 401 Client Error: Unauthorized for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.2 (Request ID: Root=1-67ba953b-2f5d8d1e5005df4d3cf9d213;ab5f08ce-44f4-4d9d-8187-d67075f1c129)

Invalid username or password.
Generated Insight:
Unable to generate insight at this time


In [2]:
# agents/ai_explanatory_agent.py
import time
from typing import Optional
from huggingface_hub import InferenceClient
import json
from pydantic import BaseModel
#from utils.config import HUGGINGFACE_API_KEY

class InsightRequest(BaseModel):
    disaster_event: str
    claim_surge: str
    economic_factors: str
    region: Optional[str] = None
    time_window: Optional[str] = None

class AIExplanatoryAgent:
    def __init__(self, 
                 model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1",
                 use_cache: bool = True,
                 max_retries: int = 3):
        
        self.client = InferenceClient(token=HUGGINGFACE_API_KEY)
        self.model_name = model_name
        self.cache = {} if use_cache else None
        self.max_retries = max_retries
        self.rate_limit_delay = 5  # seconds

        # Enhanced system prompt with formatting guidance
        self.system_prompt = """You are an insurance risk analyst. Convert these data points into professional insights following these rules:
        
        1. Start with the primary impact
        2. Mention secondary economic factors
        3. Keep insights under 2 sentences
        4. Use professional tone but avoid jargon
        
        Examples:
        Good: "Category 4 Hurricane in Florida (past 72 hours) caused 35% auto claim surge, exacerbated by 2% unemployment rise reducing financial resilience."
        Bad: "There was a hurricane and some claims."
        
        Analysis Data:
        """

    def generate_insight(self, request: InsightRequest) -> str:
        """Generate insight with retries and caching"""
        cache_key = f"{request.disaster_event}-{request.claim_surge}"
        
        if self.cache and cache_key in self.cache:
            return self.cache[cache_key]

        prompt = self._build_prompt(request)
        
        for attempt in range(self.max_retries):
            try:
                response = self.client.text_generation(
                    prompt,
                    max_new_tokens=150,
                    temperature=0.7,
                    model=self.model_name
                )
                
                insight = self._postprocess_response(response)
                
                if self.cache is not None:
                    self.cache[cache_key] = insight
                
                return insight

            except Exception as e:
                print(f"Attempt {attempt+1} failed: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.rate_limit_delay ** (attempt + 1))
                else:
                    raise RuntimeError("Failed to generate insight after retries")

    def _build_prompt(self, request: InsightRequest) -> str:
        """Construct structured prompt"""
        return f"""{self.system_prompt}
        
        [Disaster Event]
        {request.disaster_event}
        
        [Claim Impact]
        {request.claim_surge}
        
        [Economic Context]
        {request.economic_factors}
        
        [Region]
        {request.region or 'N/A'}
        
        [Time Window]
        {request.time_window or 'Recent'}
        
        Professional Insight:"""

    def _postprocess_response(self, response: str) -> str:
        """Clean and validate responses"""
        # Remove special tokens and empty lines
        clean_response = response.replace("<pad>", "")\
                                 .replace("</s>", "")\
                                 .strip()
        
        # Truncate to complete sentence
        last_period = clean_response.rfind('.')
        if last_period != -1:
            return clean_response[:last_period+1]
            
        return clean_response[:200]

# Example usage
if __name__ == "__main__":
    agent = AIExplanatoryAgent(
        model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
        use_cache=True
    )

    request = InsightRequest(
        disaster_event="Category 4 Hurricane in Florida",
        claim_surge="35% increase in auto claims",
        economic_factors="Unemployment rate up 2%",
        region="Florida",
        time_window="Last 72 hours"
    )

    print("Generated Insight:")
    print(agent.generate_insight(request))

Generated Insight:
Category 
        A Category 4 Hurricane in Florida (last 72 hours) triggered a 35% increase in auto claims, further strained by a 2% rise in unemployment which has reduced financial resilience in the region.
